<a href="https://colab.research.google.com/github/ssundar6087/practically_building_an_image_classifier/blob/main/02_the_data_strikes_back.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
path.ls()

In [ ]:
fname = (path/"images").ls()[0]
fname.name

In [ ]:
files = get_image_files(path/"images")
len(files)

In [ ]:
def labeler(f):
  return f[0].isupper()

In [ ]:
cats = len([x for x in files if labeler(x.name)])
dogs = len([x for x in files if not labeler(x.name)])

In [ ]:
cats, dogs, cats + dogs

In [ ]:
(cats / (cats + dogs)) * 100, (dogs / (dogs + cats)) * 100

In [ ]:
dls = ImageDataLoaders.from_name_func(path, files, labeler, item_tfms=Resize(224))

## Verify Dataloading
- Cats = True
- Dogs = False

In [ ]:
dls.show_batch(nrows=3,ncols=6)

In [ ]:
def class_counter(dset):
  cat_cntr = len([l.item() for _, l in dset if l.item() == 1])
  dog_cntr = len([l.item() for _, l in dset if l.item() == 0])
  return cat_cntr, dog_cntr


In [ ]:
val_ds = dls.valid_ds
train_ds = dls.train_ds
train_cats, train_dogs = class_counter(train_ds)
val_cats, val_dogs = class_counter(val_ds)

print(train_cats)
print(train_dogs)
print(val_cats)
print(val_dogs)
print(train_cats + val_cats)
print(train_dogs + val_dogs)
print(train_cats / (train_cats + val_cats))
print(train_dogs / (train_dogs + val_dogs))
print(val_cats / (train_cats + val_cats))
print(val_dogs / (train_dogs + val_dogs))

## Baseline Scores with a Simple Model

In [ ]:
learn = vision_learner(dls, resnet18, pretrained=False, metrics=[error_rate, accuracy, Precision(), Recall()])

In [ ]:
images, labels = dls.one_batch()

In [ ]:
preds, _ = learn.get_preds(dl=[(images, labels)])

In [ ]:
preds[0], preds[0].sum()

In [ ]:
learn.fit(n_epoch=20, lr=1e-3, cbs=ShowGraphCallback())

### Interpret Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(16)

## Pretrained Model - Transfer Learning

In [ ]:
pre_learn = vision_learner(dls, resnet18, metrics=[error_rate, accuracy, Precision(), Recall()])

In [ ]:
pre_learn.fine_tune(epochs=20, base_lr=1e-3, cbs=ShowGraphCallback())

### Interpret Results

In [ ]:
interp = ClassificationInterpretation.from_learner(pre_learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(16)

## Cleaning Data

In [ ]:
from fastai.vision.widgets import *

In [ ]:
cleaner = ImageClassifierCleaner(pre_learn)

In [ ]:
cleaner

## App

In [ ]:
uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
out = "cat" if is_cat == True else "dog"
prob = probs[1].item() if is_cat == True else probs[0].item()
print(f"It's a {out} with probability {prob:.3f}")


In [ ]:
probs, is_cat